In [68]:
from typing import List, Tuple
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import pandas as pd
import base64
from io import BytesIO
from matplotlib import pyplot as plt 

outputs = [
    "1000000000",  # 1
    "0100000000",  # 2
    "0010000000",  # 3
    "0001000000",  # 4
    "0000100000",  # 5
    "0000010000",  # 6
    "0000001000",  # 7
    "0000000100",  # 8
    "0000000010",  # 9
    "0000000001",  # 0
]

In [69]:
class NeuralNetwork(nn.Module):
    def __init__(self, nnets: int):
        super(NeuralNetwork, self).__init__()
        # first layer nnets neurons and tanh activation
        self.fc1 = nn.Linear(20, nnets)
        self.fc1.activation = nn.Tanh()
        # second layer 10 neurons and linear activation
        self.fc2 = nn.Linear(nnets, 10)

        # the exit from the last layer a number between 0 and 1
        self.fc2.activation = nn.Sigmoid()

        self.model = nn.Sequential(self.fc1, self.fc2)

    

    def forward(self, x):
        x = self.model(x)
        return x

In [70]:
def read_tiles_from_image(image_path: str) -> List[List[int]]:
    # open the image "numeros.png"
    img = Image.open(image_path)
    # the image is a 40 x 5 image
    # split the image into 10 4 x 5 images

    # convert the image to black and white
    img = img.convert("1")

    numbers = []
    for i in range(10):
        # crop the image
        numbers.append(img.crop((i * 4, 0, (i + 1) * 4, 5)))

    # create a list of bit strips of the numbers
    # the index of the list is the number
    numbers_list = []
    for i in range(10):
        numbers_list.append([])
        for j in range(5):
            for k in range(4):
                numbers_list[i].append(0 if numbers[i].getpixel((k, j)) == 255 else 1)

    print("Pixels of the numbers in the image:")
    for i in numbers_list:
        print("".join(str(x) for x in i))

    return numbers_list

In [71]:
def bits_to_img(bits: List[int]) -> Image:
    img = Image.new("1", (4, 5))
    for i in range(5):
        for j in range(4):
            img.putpixel(
                (
                    j,
                    i,
                ),
                255 if bits[i * 4 + j] == 0 else 0,
            )
    # resize the image to 40 x 50
    img = img.resize((20, 25))
    return img


def img_to_base64(img: Image) -> str:
    with BytesIO() as buffer:
        img.save(buffer, "PNG")
        return base64.b64encode(buffer.getvalue()).decode()


def image_formatter(base64img: str) -> str:
    return f'<img style="display:inline-block; width: 30px; height: 30px; margin: 0px; image-rendering: pixelated;" src="data:image/png;base64,{base64img}">'


def bits_tensor_to_printable(bits: List[int]) -> str:
    img = bits_to_img(bits)
    base64img = img_to_base64(img)
    return image_formatter(base64img)

In [72]:
def train_network(
    input_tensors: torch.Tensor,
    output_tensors: torch.Tensor,
    nnets: int,
    epochs: int,
    goal: float,
    learning_rate: float,
    momentum: float,
) -> Tuple[NeuralNetwork, pd.DataFrame]:
    # Mean Squared Loss
    criterion = nn.MSELoss()
    model = NeuralNetwork(nnets)
    # stochastic gradient descent
    optimizer = optim.SGD(  # Stochastic Gradient Descent/Gradient Descent with Momentum (TRAINGDM Equivalent)
        model.parameters(), lr=learning_rate, momentum=momentum
    )

    history_data = pd.DataFrame(
        columns=["nnets", "epoch", "goal", "learning_rate", "momentum"]
    )

    history_data = pd.concat(
        [
            history_data,
            pd.DataFrame(
                [{
                    "nnets": nnets,
                    "epoch": 0,
                    "goal": goal,
                    "learning_rate": learning_rate,
                    "momentum": momentum,
                }]
            ),
        ],
        ignore_index=True,
    )
    print(f"""nnets {nnets}
epochs {epochs}
goal {goal}
learning_rate {learning_rate}
momentum {momentum}""")

    # Train the neural network
    for epoch in range(epochs):
        # Forward pass
        outputs = model(input_tensors)

        loss = criterion(outputs, output_tensors)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        history_data = pd.concat(
            [
                history_data,
                pd.DataFrame(
                    [{
                        "nnets": nnets,
                        "epoch": epoch,
                        "goal": goal,
                        "learning_rate": learning_rate,
                        "momentum": momentum,
                    }]
                ),
            ],
            ignore_index=True,
        )

        if epoch % 1000 == 0:
            print(
                f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.6f}, Learning Rate: {learning_rate}, Momentum: {momentum}"
            )

        if loss.item() < goal:
            break

    print(f"Final loss: {loss.item():.6f}")
    return model, history_data


In [73]:
def flip_n_bits(bits: torch.Tensor, n: int = 1) -> torch.Tensor:
    import random
    # seed random number generator
    random.seed(1)
    randomIdxs = random.sample(range(0, len(bits)), n)
    for i in randomIdxs:
        bits[i] = 1 if bits[i] == 0 else 0
    return bits

In [74]:
def test_network(
    model: NeuralNetwork,
    input_tensors: List[torch.Tensor],
    expected_output_tensors: List[torch.Tensor],
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    # make a copy of the original input tensors
    original_input_tensors = input_tensors.clone()

    # create a dataframe to store the test results
    test_results_0 = pd.DataFrame(
        columns=[
            "input",
            "correct_output",
            "prediction",
            "is_correct",
            "n_wrong_bits",
        ]
    )
    test_results_1 = pd.DataFrame(
        columns=[
            "input",
            "correct_output",
            "prediction",
            "is_correct",
            "n_wrong_bits",
        ]
    )
    test_results_2 = pd.DataFrame(
        columns=[
            "input",
            "correct_output",
            "prediction",
            "is_correct",
            "n_wrong_bits",
        ]
    )
    test_results_3 = pd.DataFrame(
        columns=[
            "input",
            "correct_output",
            "prediction",
            "is_correct",
            "n_wrong_bits",
        ]
    )

    # TODO: find a better name to this function
    def this_test(
        model: NeuralNetwork,
        input_tensor: torch.Tensor,
        correct_output: torch.Tensor,
        test_results: pd.DataFrame,
        n_wrong_bits: int,
    ):
        with torch.no_grad():
            input_tensor = flip_n_bits(input_tensor, n_wrong_bits)
            outputs: torch.Tensor = model(input_tensor)
            outputs = outputs * 10
            sigmoid = nn.Sigmoid()
            outputs = sigmoid(outputs)

            # check if the greatest value is in the correct position
            outputs_pos = torch.argmax(outputs)
            correct_output_pos = torch.argmax(correct_output)
            
            is_correct = outputs_pos == correct_output_pos

            test_results = pd.concat(
                [
                    test_results,
                    pd.DataFrame(
                        [
                            {
                                "input": input_tensor,
                                "correct_output": correct_output,
                                "prediction": outputs_pos + 1,
                                "is_correct": "Sim" if is_correct else f"Não ({outputs_pos + 1})",
                                "n_wrong_bits": n_wrong_bits,
                            }
                        ]
                    ),
                ],
                ignore_index=True,
            )

        return test_results

    # Test the neural network
    print("Testing the neural network")
    for i in range(len(input_tensors)):
        test_results_0 = this_test(
            model,
            input_tensors[i],
            expected_output_tensors[i],
            test_results_0,
            0,
        )

    input_tensors = original_input_tensors.clone()
    # flip 1 bit in each input and test again
    print("Testing the neural network with 1 bit flipped in each input")
    for i in range(len(input_tensors)):
        test_results_1 = this_test(
            model,
            input_tensors[i],
            expected_output_tensors[i],
            test_results_1,
            1,
        )

    input_tensors = original_input_tensors.clone()
    # flip 2 bits in each input and test again
    print("Testing the neural network with 2 bits flipped in each input")
    for i in range(len(input_tensors)):
        test_results_2 = this_test(
            model,
            input_tensors[i],
            expected_output_tensors[i],
            test_results_2,
            2,
        )

    input_tensors = original_input_tensors.clone()
    # flip 3 bits in each input and test again
    print("Testing the neural network with 3 bits flipped in each input")
    for i in range(len(input_tensors)):
        test_results_3 = this_test(
            model,
            input_tensors[i],
            expected_output_tensors[i],
            test_results_3,
            3,
        )

    return test_results_0, test_results_1, test_results_2, test_results_3


In [89]:
def tests(
    input_tensors: torch.Tensor, output_tensors: torch.Tensor
) -> Tuple[List[pd.DataFrame], List[pd.DataFrame]]:
    train_params = [
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.4, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.4},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.4},
    ]
    neural_networks = [15, 25, 35]

    train_history_data_frames: List[pd.DataFrame] = []

    test_results_data_frames: List[pd.DataFrame] = []

    for nnets in neural_networks:
        for params in train_params:
            epochs = params["epochs"]
            goal = params["goal"]
            learning_rate = params["learning_rate"]
            momentum = params["momentum"]

            # Train the neural network
            trained_model, model_history = train_network(
                input_tensors,
                output_tensors,
                nnets,
                epochs,
                goal,
                learning_rate,
                momentum,
            )
            train_history_data_frames.append(model_history)

            # Test the neural network
            (
                test_results_0,
                test_results_1,
                test_results_2,
                test_results_3,
            ) = test_network(trained_model, input_tensors, output_tensors)
            test_results_data_frames.append(test_results_0)
            test_results_data_frames.append(test_results_1)
            test_results_data_frames.append(test_results_2)
            test_results_data_frames.append(test_results_3)

    return train_history_data_frames, test_results_data_frames

In [76]:
data = read_tiles_from_image("numeros.png")
len(data)

Pixels of the numbers in the image:
01001100010001001110
01101001001001001111
11100001001000011110
10101010111100100010
11111000111000011110
01111000111010010110
11110001001001000100
01101001011010010110
01101001011100011111
01101001100110010110


10

In [77]:
input_tensor = torch.tensor(data, dtype=torch.float)
input_tensor 
len(input_tensor)

10

In [78]:
correct_output = torch.tensor(
    [list(map(int, outp)) for outp in outputs], dtype=torch.float
)
correct_output

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [90]:
train_history_data_frames, test_results_data_frames = tests(
    input_tensor, correct_output
)

nnets 15
epochs 10000
goal 0.0005
learning_rate 0.1
momentum 0.0
Epoch [0/10000], Loss: 0.149054, Learning Rate: 0.1, Momentum: 0.0
Epoch [1000/10000], Loss: 0.008809, Learning Rate: 0.1, Momentum: 0.0
Epoch [2000/10000], Loss: 0.002509, Learning Rate: 0.1, Momentum: 0.0
Final loss: 0.000499
Testing the neural network
Testing the neural network with 1 bit flipped in each input
Testing the neural network with 2 bits flipped in each input
Testing the neural network with 3 bits flipped in each input
nnets 15
epochs 10000
goal 0.0005
learning_rate 0.4
momentum 0.0
Epoch [0/10000], Loss: 0.222768, Learning Rate: 0.4, Momentum: 0.0
Final loss: 0.000498
Testing the neural network
Testing the neural network with 1 bit flipped in each input
Testing the neural network with 2 bits flipped in each input
Testing the neural network with 3 bits flipped in each input
nnets 15
epochs 10000
goal 0.0005
learning_rate 0.9
momentum 0.0
Epoch [0/10000], Loss: 0.140657, Learning Rate: 0.9, Momentum: 0.0
Fina

In [80]:
def tensor_to_string(tensor: torch.Tensor) -> str:
    return ",".join(map(str, map(lambda x: round(float(x)), tensor)))

In [93]:
test_results_data_frames[0] 

,input,correct_output,prediction,is_correct,n_wrong_bits
0,"[tensor(0.), tensor(1.), tensor(0.), tensor(0....","[tensor(1.), tensor(0.), tensor(0.), tensor(0....",tensor(1),Sim,0
1,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(1.), tensor(0.), tensor(0....",tensor(2),Sim,0
2,"[tensor(1.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(1.), tensor(0....",tensor(3),Sim,0
3,"[tensor(1.), tensor(0.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(1....",tensor(4),Sim,0
4,"[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(5),Sim,0
5,"[tensor(0.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(6),Sim,0
6,"[tensor(1.), tensor(1.), tensor(1.), tensor(1....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(7),Sim,0
7,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(8),Sim,0
8,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(9),Sim,0
9,"[tensor(0.), tensor(1.), tensor(1.), tensor(0....","[tensor(0.), tensor(0.), tensor(0.), tensor(0....",tensor(10),Sim,0


In [95]:
all_results = ""

train_params = [
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.4, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.0},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.1, "momentum": 0.4},
        {"epochs": 10000, "goal": 0.0005, "learning_rate": 0.9, "momentum": 0.4},
    ]
neural_networks = [15, 25, 35]
for index,nnets in enumerate(neural_networks):
    for index2, params in enumerate(train_params):
        pos = (index * len(train_params) + index2) * 4
        title = f"Neural network com {nnets} neurons, {params['epochs']} epochs, {params['goal']} goal, {params['learning_rate']} learning rate, {params['momentum']} momentum"
        zero_wrong_bits = test_results_data_frames[pos]
        one_wrong_bits = test_results_data_frames[pos + 1]
        two_wrong_bits = test_results_data_frames[pos + 2]
        three_wrong_bits = test_results_data_frames[pos + 3]
        data_frame = pd.DataFrame(
            [
            ]
        )

        data_frame["Entrada"] = zero_wrong_bits["input"].apply(lambda x: bits_tensor_to_printable(x))
        data_frame["(um bit errado)"] = one_wrong_bits["input"].apply(lambda x: bits_tensor_to_printable(x))
        data_frame["(dois bits errados)"] = two_wrong_bits["input"].apply(lambda x: bits_tensor_to_printable(x))
        data_frame["(Tres bits errados)"] = three_wrong_bits["input"].apply(lambda x: bits_tensor_to_printable(x))
        data_frame["Esta correto"] = zero_wrong_bits["is_correct"]
        data_frame["Esta correto um bit errado"] = one_wrong_bits["is_correct"]
        data_frame["Esta correto dois bits errados"] = two_wrong_bits["is_correct"]
        data_frame["Esta correto Tres bits errados"] = three_wrong_bits["is_correct"] 
        # transpose the dataframe
        data_frame = data_frame.T
        html_code = data_frame.to_html(escape=False)
        all_results += f"<h1>{title}</h1>"
        all_results += html_code

#save 
with open("results.html", "w") as file:
    file.write(all_results)


HTML(all_results)


,0,1,2,3,4,5,6,7,8,9
Entrada,,,,,,,,,,
(um bit errado),,,,,,,,,,
(dois bits errados),,,,,,,,,,
(Tres bits errados),,,,,,,,,,
Esta correto,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim
Esta correto um bit errado,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim
Esta correto dois bits errados,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim,Sim
Esta correto Tres bits errados,Sim,Sim,Sim,Sim,Não (6),Sim,Não (3),Sim,Sim,Sim
,0,1,2,3,4,5,6,7,8,9
Entrada,,,,,,,,,,
